In [ ]:
import gumbo_client

client = gumbo_client.Client(username="szhang")
pr_table = client.get("omics_profile")
pr_table = pr_table.set_index("profile_id")

In [ ]:
# pr_table.loc[pr_table['public_release_date'].notnull(), 'public_release_date'] = "22Q2"
# pr_table.loc[pr_table['consortium_release_date'].notnull(), 'consortium_release_date'] = "22Q2"
# pr_table.loc[pr_table['internal_release_date'].notnull(), 'internal_release_date'] = "22Q2"
# pr_table.loc[pr_table['ibm_release_date'].notnull(), 'ibm_release_date'] = "22Q2"

In [ ]:
dates = pr_table[['public_release_date', 'consortium_release_date', 'internal_release_date', 'ibm_release_date']]
dates

In [ ]:
dates['no_of_nans'] = dates.isnull().sum(axis=1)
prs_to_backfill = dates[(dates['no_of_nans'] != 4) & (dates['no_of_nans'] != 0)].index.tolist()
len(prs_to_backfill)

In [ ]:
import json
from depmapomics.config import *

rna_renaming = {}
with open('../temp/22Q2/rna_sample_renaming.json') as json_file:
    rna_renaming = json.load(json_file)
rna_renaming_swapped = dict([(value, key) for key, value in rna_renaming.items()])

wgs_renaming = {}
wes_renaming = {}

with open('../temp/22Q2/wgs_sample_renaming.json') as json_file:
    wgs_renaming = json.load(json_file)

with open('../temp/22Q2/wes_sample_renaming.json') as json_file:
    wes_renaming = json.load(json_file)
    
# combine renaming dictionaries, prioritizing WES
wes_renaming.update(RESCUE_FOR_MUTATION_WES)
wgs_renaming.update(RESCUE_FOR_MUTATION_WGS)
wgs_renaming_swapped = dict([(value, key) for key, value in wgs_renaming.items()])
wes_renaming_swapped = dict([(value, key) for key, value in wes_renaming.items()])

In [ ]:
def compare_quarters(dataset_old, dataset_new, newq, portal):
    exp_old = tc.get(name=dataset_old, file='CCLE_expression')
    exp_new = tc.get(name=dataset_new, file='CCLE_expression')
    cn_old = tc.get(name=dataset_old, file='CCLE_gene_cn')
    cn_new = tc.get(name=dataset_new, file='CCLE_gene_cn')
    new_exp_arxspans = set(exp_new.index).difference(set(exp_old.index))
    new_cn_arxspans = set(cn_new.index).difference(set(cn_old.index))
    
    rna_seqids = [rna_renaming_swapped.get(key) for key in new_exp_arxspans]
    wgs_seqids = [wgs_renaming_swapped.get(key) for key in new_cn_arxspans]
    
    rna_prids = pr_table[pr_table.main_sequencing_id.isin(rna_seqids)].index.tolist()
    wgs_prids = pr_table[pr_table.main_sequencing_id.isin(wgs_seqids)].index.tolist()
    
    prs = rna_prids + wgs_prids
    prids = [pr for pr in prs if pr in prs_to_backfill]
    
    pr_table.loc[prids, portal] = newq

In [ ]:
# 21Q2
PREV_VIRTUAL['public'] = 'public-21q2-110d'
PREV_VIRTUAL['ibm'] = 'ibm-21q2-9ed1'
PREV_VIRTUAL['dmc'] = 'dmc-21q2-27e1'
PREV_VIRTUAL['internal'] = 'internal-21q2-9d16'

# 21Q3
PREV_VIRTUAL["public"] = "public-21q3-bf1e"
PREV_VIRTUAL["ibm"] = "ibm-21q3-179f"
PREV_VIRTUAL["dmc"] = "dmc-21q3-482c"
PREV_VIRTUAL["internal"] = "internal-21q3-fe4c"

# 21Q4
PREV_VIRTUAL["public"] = "public-21q4-a0d6"
PREV_VIRTUAL["ibm"] = "ibm-21q4-4e18"
PREV_VIRTUAL["dmc"] = "dmc-21q4-5725"
PREV_VIRTUAL["internal"] = "internal-21q4-ac0a"

# 22Q1
PREV_VIRTUAL["public"] = "public-22q1-305b"
PREV_VIRTUAL["ibm"] = "ibm-22q1-cce1"
PREV_VIRTUAL["dmc"] = "dmc-22q1-d00a"
PREV_VIRTUAL["internal"] = "internal-22q1-1778"

In [ ]:
compare_quarters("internal-22q1-1778", "internal-22q2-097a", "2022-05-01", "internal_release_date")
compare_quarters("internal-21q4-ac0a", "internal-22q1-1778", "2022-02-01", "internal_release_date")
compare_quarters("internal-21q3-fe4c", "internal-21q4-ac0a", "2021-11-01", "internal_release_date")
compare_quarters('internal-21q2-9d16', "internal-21q3-fe4c", "2021-08-01", "internal_release_date")

In [ ]:
compare_quarters("ibm-22q1-cce1", "ibm-22q2-a71e", "2022-05-01", "ibm_release_date")
compare_quarters("ibm-21q4-4e18", "ibm-22q1-cce1", "2022-02-01", "ibm_release_date")
compare_quarters("ibm-21q3-179f", "ibm-21q4-4e18", "2021-11-01", "ibm_release_date")
compare_quarters('dmc-21q2-27e1', "ibm-21q3-179f", "2021-08-01", "ibm_release_date")

In [ ]:
compare_quarters("dmc-22q1-d00a", 'dmc-22q2-5e51', "2022-05-01", "consortium_release_date")
compare_quarters("dmc-21q4-5725", "dmc-22q1-d00a", "2022-02-01", "consortium_release_date")
compare_quarters("dmc-21q3-482c", "dmc-21q4-5725", "2021-11-01", "consortium_release_date")
compare_quarters('dmc-21q2-27e1', "dmc-21q3-482c", "2021-08-01", "consortium_release_date")

In [ ]:
pr_table = pr_table.replace("22Q2", "2021-05-01")
pr_table = pr_table.reset_index(level=0)
pr_table

In [ ]:
client.update("omics_profile", pr_table)

In [ ]:
client.commit()

In [ ]:
pr_table[['public_release_date', 'consortium_release_date', 'internal_release_date', 'ibm_release_date']]